<a href="https://colab.research.google.com/github/pndang/MATH189/blob/main/Discussion_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This notebook may contain mistakes. Please use it cautiously and write your own code and explanation for your homework.

In [ ]:
import pandas as pd
import numpy as np

# Seed for reproducibility
np.random.seed(42)

# Generate toy dataset
data = pd.DataFrame({
    'group': ['female'] * 100 + ['male'] * 100,
    'value': np.concatenate([np.random.normal(70, 10, 100), np.random.normal(75, 15, 100)])
})

data.head()

,group,value
0,female,74.967142
1,female,68.617357
2,female,76.476885
3,female,85.230299
4,female,67.658466


In [ ]:
female_data = data[data['group'] == 'female']['value']
male_data = data[data['group'] == 'male']['value']
len(female_data), len(male_data)

(100, 100)

## Confidence Intervals for difference of two means

The code snippet uses the `statsmodels` library to compute the confidence interval of the difference between the means of two independent samples. This is particularly useful in understanding how different the two groups are, with a specified level of confidence. Here's a breakdown of the code and its components:

1. **Creating `CompareMeans` object**:
   - `sms.CompareMeans(sms.DescrStatsW(female_data), sms.DescrStatsW(male_data))` creates a `CompareMeans` object. This object is used to compare two sets of data.
   - `sms.DescrStatsW` is a class for descriptive statistics with weights. In this context, weights are assumed to be 1 for all data points, effectively calculating unweighted statistics.

2. **`tconfint_diff` method**:
   - `conf_int = cm.tconfint_diff(alpha=0.01, alternative='two-sided', usevar='unequal')`
   - `alpha=0.01`: This specifies the confidence level for the interval. With `alpha=0.01`, we're asking for a 99% confidence interval. It's calculated as `1 - alpha`, so in this case, `1 - 0.01 = 0.99` or 99%.
   - `alternative='two-sided'`: This indicates that we're interested in differences in both directions, meaning we're testing if the first mean is either greater than or less than the second mean.
   - `usevar='unequal'`: This argument specifies that the method should assume unequal variances in the two samples. This is important for the Welch's t-test, which does not assume equal population variances.

In summary, this code calculates the 99% confidence interval for the difference between the means of two samples, assuming they have unequal variances, and considers both possible directions of difference. This approach is generally more conservative and is suitable for a wide range of applications, especially when the sample sizes or variances are different.


In [ ]:
female_data.var(), male_data.var()

(82.476989363016, 204.6340118386687)

In [ ]:
import statsmodels.stats.api as sms

cm = sms.CompareMeans(sms.DescrStatsW(female_data), sms.DescrStatsW(male_data))
conf_int = cm.tconfint_diff(alpha=0.01, alternative='two-sided', usevar='unequal')  # 99% CI
print(f"99% CI for the difference in means: {conf_int}")

99% CI for the difference in means: (-10.787837466759164, -1.958230492620424)


## Two-sample t-test

### Function Overview

- `stats.ttest_ind` is a function from the SciPy library that conducts a t-test for the means of two independent samples. This test assumes that the populations have identical variances by default.

### Arguments Explained

- `male_data, female_data`: These are the two arrays of sample observations. They must be independent of each other, meaning that the samples come from two distinct groups.
- `equal_var=False`: This argument specifies whether the variance of the two groups should be assumed to be equal. When set to `False`, it performs Welch's t-test, which does not assume equal population variance. This is appropriate when the two samples have different variances and possibly different sample sizes. If set to `True`, a standard independent 2 sample t-test is performed assuming equal population variances.

### Other Possible Options

- `alternative`: This argument specifies the alternative hypothesis. The options are:
  - `'two-sided'`: Tests against the hypothesis that the means are not equal. This is the default behavior.
  - `'less'`: Tests against the hypothesis that the mean of the first group is less than the mean of the second group.
  - `'greater'`: Tests against the hypothesis that the mean of the first group is greater than the mean of the second group.
  Choosing the correct `alternative` is crucial for the interpretation of the test. It should be selected based on the specific research question or hypothesis being tested.

### Output Explained

- `T-statistic`: This is the calculated t-statistic value. It represents the degree to which the groups differ, in terms of the number of standard errors. A larger absolute value of the t-statistic indicates a greater difference between the groups.
- `P-value`: This value indicates the probability of observing the data (or something more extreme) if the null hypothesis is true. A small p-value (typically ≤ 0.05) indicates strong evidence against the null hypothesis, so you reject the null hypothesis.


In [ ]:
from scipy import stats

t_stat, p_value = stats.ttest_ind(male_data, female_data, equal_var=False)

# Printing the results
print(f"T-statistic: {t_stat}")
print(f"P-value: {p_value}")

T-statistic: 3.7611557434761536
P-value: 0.00023342322674345948


In [ ]:
alpha = 0.01  # Significance level
if p_value < alpha:
    print("Reject the null hypothesis: There is a significant difference in mean of 'value' between males and females.")
else:
    print("Fail to reject the null hypothesis: There is no significant difference in mean of 'value' between males and females.")

Reject the null hypothesis: There is a significant difference in mean of 'value' between males and females.


# Chi-Square Tests Overview

The chi-square test is a statistical method used to compare categorical data. Despite the different types of chi-square tests, the core idea remains the same: to evaluate how well the observed data fit an expected distribution. The general chi-square test statistic is defined as:

$$ \chi^2 = \sum \frac{(O - E)^2}{E} $$

Where:
- $O$ represents the observed frequency counts.
- $E$ represents the expected frequency counts under the null hypothesis.

The expected counts are determined differently depending on the type of test: goodness of fit, independence, or homogeneity. Below we explore these three tests in more detail.


## Test for Goodness of Fit

The test for goodness of fit is applied when you have one categorical variable from a single population and want to compare the observed counts to expected counts that have been predetermined or are theoretically expected.

- **Null Hypothesis** ($H_0$): The observed distribution fits the expected distribution $p_i$.
- **Alternative Hypothesis** ($H_a$): The observed distribution does not fit the expected distribution $p_i$.

The expected counts $E$ are calculated based on the preset proportions for each category. If the expected proportion for a category is $p_i$, and there are $n$ total observations, then the expected count for that category is: $$ E_i = n \times p_i.$$


## Test for Independence

The test for independence is used to determine whether there is a significant association between two categorical variables in a single population. It is typically represented in a contingency table format.

- **Null Hypothesis** ($H_0$): The two variables are independent.
- **Alternative Hypothesis** ($H_a$): The two variables are not independent.

To calculate the expected counts $E$, we use the marginal totals of the contingency table. The observed count $O_{ij}$ is the number in the cell at the intersection of the $i$-th row and $j$-th column. The expected count $E_{ij}$ is calculated as:

$$ E_{ij} = \frac{(\text{row total}_i \times \text{column total}_j)}{\text{grand total}} $$

Here is an example of a 2x2 contingency table:

|               | Category A | Category B | Row Total |
|---------------|------------|------------|-----------|
| **Group 1**   | $O_{11}$ | $O_{12}$ | $n_{1.}$ |
| **Group 2**   | $O_{21}$ | $O_{22}$ | $n_{2.}$ |
| **Col Total** | $n_{.1}$ | $n_{.2}$ | $n$     |

The expected frequency for the cell at the intersection of Group 1 and Category A would be calculated as: $$ E_{11} = \frac{n_{1.} \times n_{.1}}{n} .$$


## Test for Homogeneity

The Chi-square test for homogeneity is similar to the test for independence in terms of its application and calculation, but it is used to compare the distribution of a categorical variable across two or more populations or groups to see if they have the same distribution.

### Null Hypothesis ($H_0$):

The distribution of the categorical variable is the same across all populations or groups.

### Alternative Hypothesis ($H_1$):

At least one population or group has a different distribution of the categorical variable.

### Contingency Table for Homogeneity Test:

The contingency table for a homogeneity test is constructed similarly to the one for the independence test, with each row representing a different population or group and each column representing the categories of the variable of interest.

#### Example Contingency Table:

|               | Category A | Category B | Row Total |
|---------------|------------|------------|-----------|
| **Population 1**   | $O_{11}$ | $O_{12}$ | $n_{1.}$ |
| **Population 2**   | $O_{21}$ | $O_{22}$ | $n_{2.}$ |
| **Col Total** | $n_{.1}$ | $n_{.2}$ | $n$     |

The expected frequencies are calculated in the same way as in the test for independence, considering the row and column totals to assess the distribution across different populations or groups.


## $\chi^2$-test for homogeneity

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats

# Sample data generation
np.random.seed(42)  # For reproducibility
data = {
    'sex': np.random.choice(['Male', 'Female'], size=200),
    'education': np.random.choice(['High School', 'Bachelor', 'Master', 'PhD'], size=200)
}
df_filtered = pd.DataFrame(data)
df_filtered

,sex,education
0,Male,Master
1,Female,PhD
2,Male,Master
3,Male,High School
4,Male,PhD
...,...,...
195,Female,PhD
196,Female,PhD
197,Female,Bachelor
198,Male,PhD


In [ ]:
# Crosstab to get observed frequencies
O_table = pd.crosstab(df_filtered['sex'], df_filtered['education'])
print("Observed frequencies table:")
print(O_table)

p_table = O_table.div(O_table.sum(axis=1), axis=0)
total_proportions = O_table.sum(axis=0) / O_table.values.sum()
p_table.loc['total'] = total_proportions
print(p_table)

# Chi-squared test
chi2, p, dof, expected = stats.chi2_contingency(O_table)
print(f"\nChi-squared: {chi2}")
print(f"P-value: {p}")
print(f"Degrees of freedom: {dof}")
print("Expected frequencies table:")
print(expected)

Observed frequencies table:
education  Bachelor  High School  Master  PhD
sex                                          
Female           21           24      24   31
Male             19           25      26   30
education  Bachelor  High School  Master    PhD
sex                                            
Female         0.21        0.240    0.24  0.310
Male           0.19        0.250    0.26  0.300
total          0.20        0.245    0.25  0.305

Chi-squared: 0.21680160588825695
P-value: 0.9748324264267186
Degrees of freedom: 3
Expected frequencies table:
[[20.  24.5 25.  30.5]
 [20.  24.5 25.  30.5]]


# For manual computation:

In [ ]:
O_table = np.array(O_table)
manual_chi2 = ((O_table-expected)**2/expected).sum().sum()
manual_chi2

0.21680160588825695

In [ ]:
alpha = 0.05
if p < alpha:
  print('reject')
else:
  print('null')

null


| \ | Category 1 | Category 2 | $\dots$ | Category $k$ | Total |
|:--------:|:-------------------:|:-------------------:|:-------------------:|:-------------------:|:-------------------:|
| $X_1$ | $O_{11}$               | $O_{12}$               | $\dots$ | $O_{1k}$ | $n_1$ |
| $X_2$ | $O_{21}$               | $O_{22}$               | $\dots$ | $O_{2k}$ | $n_2$ |
| $\vdots$ | $\vdots$ | $\vdots$ | $\vdots$ | $\dots$ | $\vdots$ | $\vdots$ |
| $X_r$ | $O_{r1}$               | $O_{r2}$               | $\dots$ | $O_{rk}$ | $n_r$ |
| Total | $T_{1}$               | $T_{2}$               | $\dots$ | $T_{k}$ | $n$ |



| \ | Category 1 | Category 2 | $\dots$ | Category $k$ | Total |
|:--------:|:-------------------:|:-------------------:|:-------------------:|:-------------------:|:-------------------:|
| $X_1$ | $\hat{p}_{11} = {O_{11}}/{n_1}$               | $\hat{p}_{12} = {O_{12}}/{n_1}$               | $\dots$ | $\hat{p}_{1k} = {O_{1k}}/{n_1}$ | $n_1$ |
| $X_2$ | $\hat{p}_{21} = {O_{21}}/{n_2}$               | $\hat{p}_{22} = {O_{22}}/{n_2}$               | $\dots$ | $\hat{p}_{2k} = {O_{2k}}/{n_2}$ | $n_2$ |
| $\vdots$ | $\vdots$ | $\vdots$ | $\vdots$ | $\dots$ | $\vdots$ | $\vdots$ |
| $X_r$ | $\hat{p}_{r1} = {O_{r1}}/{n_r}$               | $\hat{p}_{r2} = {O_{r2}}/{n_r}$               | $\dots$ | $\hat{p}_{rk} = {O_{rk}}/{n_r}$ | $n_r$ |
| Total | $\hat{p}_{1} = {T_{1}}/{n}$               | $\hat{p}_{2} = {T_{2}}/{n}$               | $\dots$ | $\hat{p}_{k} = {T_{k}}/{n}$ | $n$ |

| Anatomy of the hypothesis test    |  Answer  |
|:---------------------------------:|:--------:|
| Assumption                        | $X_1 \sim \text{Multinomial}(n_1, p_{11}, p_{12}, \dots, p_{1k})$, $X_2 \sim \text{Multinomial}(n_1, p_{21}, p_{22}, \dots, p_{2k})$ |
| Population parameter              | $\theta = (p_{11}, p_{12}, \dots, p_{kr})$ where $p_{ij}$ represents the proportion of the $j$th category in the $i$th population. |
| Sample statistic                  | $\hat\theta = (\hat p_{11}, \hat p_{12}, \dots, \hat p_{kr})$ where $\hat p_{ij}$ is the observed proportion of the $j$th category in the $i$th sample. |
| Test statistic                    | $\displaystyle T = \sum_{i=1}^k \sum_{j=1}^r \frac{(O_{ij} - E_{ij})^2}{E_{ij}} \sim {\chi}^2_{(k-1) \times (r-1)}$ where $O_{ij}$ is the observed frequency and $E_{ij}$ is the expected frequency for the $j$th category in the $i$th population. |
| Null hypothesis                   | $H_0: p_{1j} = p_{2j} = \cdots = p_{kj}$ for all $j$. This states that the proportions for each category are the same across all populations. |
| Alternate hypothesis              | $H_a: p_{ij} \neq p_{i'j}$ for some $i \neq i'$ and any $j$. This suggests that at least one population proportion is different. |


 Definition of $E_{ij}$: $E_{ij} = n_i \times p_j$ where $n_i$ is the total number of observations in the $i$th sample, and $p_j$ is the overall proportion of the $j$th category across all samples combined. The overall proportion $p_j$ is calculated as the sum of all observations of the $j$th category across all groups divided by the total number of observations across all groups.
